In [1]:
import numpy as np
import pandas as pd

In [9]:
def preprocessing(path = './data/data_regression_for_task.csv'):
    data = pd.read_csv(path)
    # преобразование типов
    data['SALES'] = data['SALES'].astype('int64')
    data['STORE_SALES'] = data['STORE_SALES'].astype('int64')
    # заполним единственный пропуск в ARTICLE_GROUP
    data.loc[data['ARTICLE_GROUP'].isnull(),'ARTICLE_GROUP'] = 'WINE'
    # удалим строки с пропусками в CONTRAGENT
    data.drop(data[data['CONTRAGENT'].isnull()].index, inplace=True)
    #  удалим неинформативную категорию
    data.drop(data[data['ARTICLE_GROUP'] == 'DUNNAGE'].index, inplace=True)
    # приведем в соответствие коды и наименования 
    count_names = data.groupby('ARTICLE_CODE')['ARTICLE_NAME'].unique()
    count_names = dict(count_names.apply(lambda x: x[0]))
    data['ARTICLE_NAME'] = data['ARTICLE_CODE'].map(count_names)
    data = data.groupby(['YEAR', 'MONTH', 'CONTRAGENT', 'ARTICLE_NAME',
             'ARTICLE_GROUP'])[['SALES', 'STORE_SALES']].sum().reset_index()
    # новый временной признак 
    data['YEAR_MONTH'] = data["YEAR"].astype('str') + '_' \
                         + data["MONTH"].astype('str').apply(lambda x: x.zfill(2))
    
    data.drop(data[data['YEAR_MONTH']=='2017_04'].index, inplace=True)
    
    return data

In [10]:
data = preprocessing('./data/data_regression_for_task.csv')

In [11]:
data

,YEAR,MONTH,CONTRAGENT,ARTICLE_NAME,ARTICLE_GROUP,SALES,STORE_SALES,YEAR_MONTH
5192,2017,5,8 VINI INC,SECOLI RIPASSO DELIO VAL DOC - 750ML,WINE,330,0,2017_05
5193,2017,5,A I G WINE & SPIRITS,DOM DES FONTANELLE S/BLC - 750ML,WINE,0,4,2017_05
5194,2017,5,A I G WINE & SPIRITS,LES FONTANELLES P/NOIR - 750ML,WINE,80,5,2017_05
5195,2017,5,A I G WINE & SPIRITS,LIQUID ICE VODKA - ORGANIC - 750ML,LIQUOR,1350,0,2017_05
5196,2017,5,A I G WINE & SPIRITS,TROCADERO SPARK(BRUT) - 750ML,WINE,500,3,2017_05
...,...,...,...,...,...,...,...,...
128055,2018,2,YUENGLING BREWERY,YUENGLING PORTER 4/6 NR - 12OZ,BEER,25540,22,2018_02
128056,2018,2,Z WINE GALLERY IMPORTS LLC,A GEOFFROY CHAB - 750ML,WINE,490,0,2018_02
128057,2018,2,Z WINE GALLERY IMPORTS LLC,CH FLEUR HAUT GAUSSENS BORD SUP - 750ML,WINE,0,1,2018_02
128058,2018,2,Z WINE GALLERY IMPORTS LLC,DOM DELIANCE GIVRY PREM CRU LES COMBES - 750ML,WINE,0,1,2018_02


In [22]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [25]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)

# for train, test in tscv.split(values):
#     ...

In [29]:
for train, test in tscv.split(data[['YEAR','MONTH']]):
    print(len(train))

30717
61434
92151


In [6]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data)

ValueError: could not convert string to float: 'A VINTNERS SELECTIONS'